In [ ]:
import pandas as pd
from utils.utils import preprocessing, date_column_change

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [ ]:
data = pd.read_csv("Data/пп_станции.xlsx - Лист1.csv")
data.head()

In [ ]:
data = preprocessing(data)
data.head()

In [ ]:
data[data["Станция"] == "Нижегородская"]

In [ ]:
data.iloc[:, :3]

In [ ]:
from utils.utils import catboost_learn